In [16]:
import torch
import cv2
import yaml
from acquisition import VideoFeedReader
import time

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Model
model_2 = torch.hub.load('ultralytics/yolov5', 'yolov5n').to(device) # or yolov3-spp, yolov3-tiny, custom
model = torch.hub.load('ultralytics/yolov5', 'yolov5x').to(device) # or yolov3-spp, yolov3-tiny, custom

Using cache found in /home/sach/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.6.9 torch-1.10.1 CUDA:0 (NVIDIA GeForce RTX 3080, 10015MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


Using device: cuda
WARNING ⚠️ Python 3.7.0 is required by YOLOv5, but Python 3.6.9 is currently installed


Using cache found in /home/sach/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-30 Python-3.6.9 torch-1.10.1 CUDA:0 (NVIDIA GeForce RTX 3080, 10015MiB)

Fusing layers... 


WARNING ⚠️ Python 3.7.0 is required by YOLOv5, but Python 3.6.9 is currently installed


YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [38]:
import numpy as np
from PIL import Image

path = "data/img_5_1/img_5_1_"
nb_frames = 1500

ball_class = 32
person_class = 0

codec = cv2.VideoWriter_fourcc(*'mp4v')
# Detection stream
dec_stream = cv2.VideoWriter("detection.mp4", codec, 25.0, (1280, 720), 0)
# Segmentation stream
seg_stream = cv2.VideoWriter("segmentation.mp4", codec, 25.0, (1280, 720), 0)
# Background stream
bg_stream = cv2.VideoWriter("background.mp4", codec, 25.0, (1280, 720), 0)

inf_times = []
draw_times = []

display = False

t = time.time()
for i in range(0, nb_frames):
    t = time.time()
    img_path = path + str(i).zfill(4) + '.jpg'
    cv_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    print(time.time() - t)
    t = time.time()
    res = model_2(cv_img).xyxy[0]
    print(time.time() - t)
    t = time.time()
    res = res[(res[:, 5] == ball_class) | (res[:, 5] == person_class)]
    print(time.time() - t)
    print()

    inf_times.append(time.time() - t)
    #print("Inference time: ", inf_times[-1])
    t = time.time()

    # Create a mask for bounding boxes:
    mask = np.zeros((720, 1280), dtype=np.uint8)
    for box in res:
        mask[int(box[1]):int(box[3]), int(box[0]):int(box[2])] = 1
    seg_stream.write(mask * 255)
    bg_stream.write((1 - mask) * cv_img)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    for *xyxy, conf, cls in res:
        label = '%s %.2f' % (model.names[int(cls)], conf)
        # Draw box on image
        cv2.rectangle(img, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
    dec_stream.write(img)

    if display:
        cv2.imshow("Detecting Motion...", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    draw_times.append(time.time() - t)
    t = time.time()
cv2.destroyAllWindows()
seg_stream.release()
dec_stream.release()
bg_stream.release()

# Average inference & draw time
print("Average inference time: ", np.mean(inf_times))
print("Average draw time: ", np.mean(draw_times))
# Average FPS:
print("Average FPS: ", nb_frames / (np.sum(inf_times) + np.sum(draw_times)))

0.009241819381713867
0.007359504699707031
0.00018215179443359375



ValueError: operands could not be broadcast together with shapes (720,1280) (720,1280,3) 